In [1]:
import numpy as np
import pandas as pd
from collections import Counter

train = pd.read_csv('../input/training.csv')
test = pd.read_csv('../input/test.csv')
sub = pd.read_csv('../input/sample_submission.csv')

In [2]:
for difficulty in sorted(test['difficulty'].unique()):
    print('Difficulty: ', difficulty)
    sample = test[test['difficulty']==difficulty]
    print(len(sample), sample['ciphertext'].iloc[0][:50])

Difficulty:  1
11186 }8E U%Weuoc1'8fdsc.:L5DTtKii2RMDV6VpnST264P:[mv2o!
Difficulty:  2
10966 NME-;Xz #!#qe1d?QG./[!b7n!DJ7}4c7P4=i7!27b[y7C4ms7
Difficulty:  3
11335 4V,[[p[z7=b777!z7}77psl777zbJlbQ7Smm4niTDlU[2lk!4y
Difficulty:  4
11195 780 538832 266403 380102 515387 156801 300516 2978


In [3]:
traind = Counter(' '.join(train['text'].astype(str).values))
traind = ''.join([c for c, v in traind.most_common(85)])
print(repr(traind))

test1 = test[test['difficulty']==1].reset_index(drop=True)
testd = Counter(' '.join(test1['ciphertext'].astype(str).values))
testd = ''.join([c for c, v in testd.most_common(85)])
print(repr(testd))

#Solution 1 -> Apply to difficulty 2
#Some quick adjustments based on lookups. Not really cryptography but helps to move forward :)
traind = ' etaoisnrhlducmfygwpb.v,kI\'TA"-SEBMxCHDjW)(RLONPGF!Jzq01?KVY:9U2*/3;58476ZQX%$}#@={[]'
testd =  '7lx4v!o2Q[O=y,CzV:}dFX#(Wak/qbne *JAmKp{fc6DGZj\'Tg9"YHS]Ei5)8h1MINwP@s?U3;0%$-rLuBRt.'

SubEnc_ = str.maketrans(testd, traind)

test1['text'] = test1['ciphertext'].map(lambda x: str(x).translate(SubEnc_)).values
test1.head()

' etaoisnrhlducmfygwpb.v,kI\'T"-ASBMxHCjDW)(ERLONP!FGJzq01?KVY:9U2*/;358476ZQX$%=#][}@{'
'7lx4v!o2Q[O=y,CzV:}dFX#(Wak/qbne *JAmKp{fc6DGZj\'Tg9"YHS]Ei5)8h1MINwP@s?U3;0%$-rLuBRt.'


,ciphertext_id,ciphertext,difficulty,text
0,ID_f6479f2ee,}8E U%Weuoc1'8fdsc.:L5DTtKii2RMDV6VpnST264P:[m...,1,w:?B7XkS]s(UP:)p8({g#VR!@jKKn}2RyEyDA0!nEa3ghC...
1,ID_20187e596,N*VV$-C${E$?H=103EH1a7Ov#l=7Gv#l7!o72l#lQ7o!Ol...,1,/Myy$%m$W?$4qdUQ6?qUI loved Love is never sile...
2,ID_f5670350b,tuey%v.HzU9#!tegGOa7Ql4OOV7O!Wl=7x[!o7z!OCX7il...,1,@]SuXo{qf7Gvi@SFLlI really liked this film. Ke...
3,ID_71621a965,"e)'K=(%{!q,xy4OOV(7!xo7[4Q=7xv7=lo,Q!Fl7}[4xo7...",1,"SYPjd,XWi""ctually, its hard to describe whats ..."
4,ID_92f5ceac5,"%z%zY Xzsbm3Cbmo15c7}%-Mu/6c#):X,K?Y9p[sf!f@%K...",1,XfXfzB.f8-C6m-CsUV( wX%2]TE(vYg.cj4zGDh8)i)5Xj...


In [ ]:
def test_contains_text(s):
    val = ''
    s = str(s)
    try:
        val = test1[test1['text'].str.contains(s, regex=False)]['ciphertext_id'].iloc[0]
    except: pass
    return val

train['ciphertext_id'] = train['text'].map(lambda x: test_contains_text(x))
test = pd.merge(test, train[['ciphertext_id', 'index']], how='left', on='ciphertext_id')
test['index'].fillna(sub['index'], inplace=True)
test['index'] = test['index'].astype(int)
test[['ciphertext_id', 'index']].to_csv('submission.csv', index=False)